In [1]:
import numpy as np
import pandas as pd

In [2]:
match = pd.read_csv('matches.csv')
delivery = pd.read_csv('deliveries.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'matches.csv'

In [ ]:
match.head()

In [ ]:
match.shape

In [ ]:
delivery.head()

Now I want to find out the runs scored by a team in every innings so for that I will use groupby.

In [ ]:
delivery.groupby(['match_id','inning']).sum()['total_runs']

You can clearly observe from the above code that we are getiing the runs scored by each team in each innings of evry match.
Now i will use reset_index to convert this into dataframe.

In [ ]:
delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()

I will save this code in a variable called as total_score_df.

In [ ]:
total_score_df = delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()
total_score_df

Now we will only take the total_runs of 1st innings. so we will ignore the 2nd innings score. Why? we will see this ahead.

In [ ]:
total_score_df = total_score_df[total_score_df['inning']==1]
total_score_df

We will merge this dataframe with match dataframe because we have all the details in the match dataframe except the runs scored by a team.

In [ ]:
match_df=match.merge(total_score_df[['match_id','total_runs']],left_on = 'id',right_on='match_id')
match_df

In [ ]:
match_df['team1'].unique()

As we all know previously there were few teams who are not the part of IPL anymore(eg.'Pune Warriors'etc) and also few teams who have twoo different names
(eg. 'Delhi Daredevils','Delhi capitals'etc.)

In [ ]:
teams = [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',
    'Kings XI Punjab',
    'Chennai Super Kings',
    'Rajasthan Royals',
    'Delhi Capitals'
]

In [ ]:
match_df['team1']=match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2']=match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['team1']=match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2']=match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

I am wring code for the match_df dataframe and I want only those teams who are part of the IPL. The teams who are part of the IPL till now are stored in a list called teams

In [ ]:
match_df = match_df[match_df['team1'].isin(teams)]
match_df = match_df[match_df['team2'].isin(teams)]

In [ ]:
match_df.shape

In [ ]:
match_df['dl_applied'].value_counts()

I want only those matches which are not affected by the rains.

In [ ]:
match_df=match_df[match_df['dl_applied']==0]

In [ ]:
match_df

Now I will take all the neccesary columns from this df and join that with the delivery df.

In [ ]:
match_df=match_df[['match_id','city','winner','total_runs']]

In [ ]:
match_df.merge(delivery,on='match_id').head(1)

we have every balls details, like the ball was bowled in which city,who was the winner of that match, target for the 2nd innings etc.

In [ ]:
delivery_df = match_df.merge(delivery,on='match_id')

Now I only need for those innings record where the inning no is 2. It will help us for our prediction.

In [ ]:
delivery_df = delivery_df[delivery_df['inning']==2]

In [ ]:
delivery_df.head()

In [ ]:
delivery.shape

In this df we have a column called as total_runs_y which gives us a details of runs scored on evry ball.Now we have to find out according to every match after each delivery what's the score.
To calculate this we have to group it according to match_id and then we have to apply the function called as cumsum on total_runs_y. we will store this value in a new column called as current score.

In [ ]:
delivery_df['current_score']=delivery_df.groupby('match_id').cumsum()['total_runs_y']

In [ ]:
delivery_df

Now we will subtract the current_score with target which will be our runs left.

In [ ]:
delivery_df['runs_left'] = delivery_df['total_runs_x']+1 - delivery_df['current_score']

In [ ]:
delivery_df

To calculate the balls_left column we have to use one mathematical formula.

In [ ]:
delivery_df['balls_left'] = 126-(delivery_df['over']*6+delivery_df['ball'])

In [ ]:
delivery_df

Now we will find out wickets left after each delivery. This will be tricky to find out but we will do it.

First we will go through the player_dismmised column and try to convert the Nan values into string of 0 after that we will convert the players name who got dismmised into string of 1.Then we will convert the column datatype into int.Then we used cumsum function on player_dismmised column while grouping it by match_id column which will give us the value of wicket lost.To calculate the wickets left we will simply subtract it by 10.

In [ ]:
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].fillna("0")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].apply(lambda x:x if x=="0" else "1")
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].astype('int')
wickets = delivery_df.groupby('match_id').cumsum()['player_dismissed'].values
delivery_df['wickets'] = 10-wickets
delivery_df.head()

In [ ]:
delivery_df.tail()

Now we will calculate current run rate column. We will calculate it by dividing the runs/overs.SInce we don't have the overs column we will take the help of balls left column.

In [ ]:
delivery_df['crr']=delivery_df['current_score']/((120-delivery_df['balls_left'])/6)

In [ ]:
delivery_df.head()

Now we will calculate Required run rate column.To calculate required run rate column we will perform run_left/overs_left.

In [ ]:
delivery_df['rrr'] = (delivery_df['runs_left']*6)/delivery_df['balls_left']

Now we will calculate the result column where we will get to know whether chasing team won or not. If they won we will represent it as 1 else 0.

In [ ]:
def result(row):
    return 1 if row['batting_team']==row['winner'] else 0

In [ ]:
delivery_df['result']=delivery_df.apply(result,axis=1)

Now we will take out the required columns from the df.

In [ ]:
final_df = delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr','result']]

In [ ]:
final_df

Now we will shuffle this df to ignore the biassedness in our result.

In [ ]:
final_df = final_df.sample(final_df.shape[0])

In [ ]:
final_df

In [ ]:
final_df.sample()

Now we will build the model. First we will split the data into train and test.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = final_df.iloc[:,:-1]
Y = final_df.iloc[:,-1]

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=7,stratify=Y)

In [ ]:
X_train

As we already know that ML works only with numbers not with strings. So we will apply One Hot Encoding to convert the strings into numbers.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

trf = ColumnTransformer(
    transformers=[
        ('trf', OneHotEncoder(sparse=False), ['batting_team', 'bowling_team', 'city'])
    ],
    remainder='passthrough'
)

Now we will create one pipeline where in first step we will have the above transformer and in the next step we will be having our Logistic regression model.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))
])

In [ ]:
pipe.fit(X_train,Y_train)

We are getting the above ERROR because we have null values. So first we will drop alll the null values and then rerun the X_train,X_test code.

In [ ]:
final_df.isnull().sum()

In [ ]:
final_df.dropna(inplace=True)

In [ ]:
final_df.isnull().sum()

In [ ]:
pipe.fit(X_train,Y_train)

In [ ]:
X.describe()

We can clearly observe that in the rrr column we have inf values which is creating the issue. So to  overcome this error we will ignore those rows where we have 0 balls left and will re run the model building code.

In [ ]:
final_df = final_df[final_df['balls_left']!=0]

In [ ]:
pipe.fit(X_train,Y_train)

In [ ]:
X_train.describe()

In [ ]:
y_pred = pipe.predict(X_test)

Now we will check the model score.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,y_pred)

Now we will convert this model into web. For that we require teams,City and Pipeline(Pickle)

In [ ]:
teams

In [ ]:
final_df['city'].unique()

In [ ]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [ ]:
pip show scikit-learn

In [ ]:
!pip install scikit-learn==1.4.1.post1

In [ ]:
pip install --upgrade scikit-learn
